In [0]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score,confusion_matrix
from sklearn.preprocessing import StandardScaler,LabelEncoder
stds=StandardScaler()
lable=LabelEncoder()
from sklearn.naive_bayes import GaussianNB
gb=GaussianNB()

In [3]:
traindata=pd.read_csv('train.csv')
traindata.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,15-11-2018 00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,15-11-2018 00:01,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,15-11-2018 00:02,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,15-11-2018 00:02,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,15-11-2018 00:02,62873,473,latest,0,0


In [4]:
itemdata=pd.read_csv('item_data.csv')
itemdata.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [5]:
viewlog=pd.read_csv('view_log.csv')
viewlog.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333.0,4557.0,32970.0
1,2018-10-15 08:58:00,android,503590.0,74788.0,7640.0
2,2018-10-15 08:58:00,android,573960.0,23628.0,128855.0
3,2018-10-15 08:58:00,android,121691.0,2430.0,12774.0
4,2018-10-15 08:58:00,android,218564.0,19227.0,28296.0


In [0]:
viewitem=pd.merge(viewlog,itemdata)

In [0]:
viewitemtrain=pd.merge(viewlog,traindata)

In [8]:
viewitemtrain.head()

,server_time,device_type,session_id,user_id,item_id,impression_id,impression_time,app_code,os_version,is_4G,is_click
0,2018-10-15 08:58:00,android,112333.0,4557.0,32970.0,6dd9544216149618da55e183f73e2b56,26-11-2018 14:37,213,latest,1,0
1,2018-10-15 08:58:00,android,112333.0,4557.0,32970.0,6ba86f2d1de0989ffbf73cc37f5825c9,26-11-2018 14:35,213,latest,1,0
2,2018-10-15 08:59:00,android,112333.0,4557.0,132775.0,6dd9544216149618da55e183f73e2b56,26-11-2018 14:37,213,latest,1,0
3,2018-10-15 08:59:00,android,112333.0,4557.0,132775.0,6ba86f2d1de0989ffbf73cc37f5825c9,26-11-2018 14:35,213,latest,1,0
4,2018-10-15 08:59:00,android,112333.0,4557.0,100092.0,6dd9544216149618da55e183f73e2b56,26-11-2018 14:37,213,latest,1,0


In [0]:
viewitemtrain["app_code"] = lable.fit_transform(viewitemtrain["app_code"])
viewitemtrain["user_id"] = lable.fit_transform(viewitemtrain["user_id"])
viewitemtrain["os_version"] = lable.fit_transform(viewitemtrain["os_version"])
viewitemtrain["is_4G"] = lable.fit_transform(viewitemtrain["is_4G"])
viewitemtrain["device_type"] = lable.fit_transform(viewitemtrain["device_type"])
viewitemtrain["session_id"] = lable.fit_transform(viewitemtrain["session_id"])
viewitemtrain["item_id"] = lable.fit_transform(viewitemtrain["item_id"])

In [0]:
x=viewitemtrain[['user_id','app_code','os_version','is_4G']]
y=viewitemtrain[['is_click']]

In [0]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [38]:
gb.fit(x_train, y_train)
y_pred_test = gb.predict(x_test)
print("Accuracy score for model ", accuracy_score(y_test, y_pred_test))
confusion_mx = pd.DataFrame(confusion_matrix(y_test, y_pred_test))
print(confusion_mx)
probs = gb.predict_proba(x_test) 
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
print("Precision score for DT model ", precision_score(y_test, y_pred_test))
print("Recall score for DT model ", recall_score(y_test, y_pred_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy score for model  0.9424725821998213
         0  1
0  1759028  0
1   107369  0
AUC: 0.55


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision score for DT model  0.0
Recall score for DT model  0.0


In [39]:
testdata=pd.read_csv('test.csv')
testdata.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G
0,a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,latest,1
1,caac14a5bf2ba283db7708bb34855760,2018-12-13 07:45:00,29656,44,latest,0
2,13f10ba306a19ce7bec2f3cae507b698,2018-12-13 07:46:00,25234,296,latest,1
3,39c4b4dc0e9701b55a0a4f072008fb3f,2018-12-13 07:47:00,22988,207,latest,1
4,bf5a572cca75f5fc67f4b14e58b11d70,2018-12-13 07:48:00,35431,242,latest,1


In [0]:
testdata["app_code"] = lable.fit_transform(testdata["app_code"])
testdata["user_id"] = lable.fit_transform(testdata["user_id"])
testdata["os_version"] = lable.fit_transform(testdata["os_version"])
testdata["is_4G"] = lable.fit_transform(testdata["is_4G"])

In [41]:
test_data_X =testdata.iloc[:,2:]
test_data_X.head()

,user_id,app_code,os_version,is_4G
0,16423,94,1,1
1,10858,39,1,0
2,9213,211,1,1
3,8391,150,1,1
4,13031,171,1,1


In [0]:
test_data_pred = gb.predict(test_data_X)
submission = pd.DataFrame({'impression_id':testdata.iloc[:,0], 'is_click':test_data_pred})
submission.to_csv('submission_gb.csv', index=False)